In [4]:
import json
import spacy

In [3]:
with open("../data/disney_reviews.json") as f:
    reviews = json.load(f)
    
nlp = spacy.load("en_core_web_md", disable=['parser', 'tagger', 'ner'])

In [5]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Apple Apple    Xxxxx True False
is be    xx True True
looking look    xxxx True False
at at    xx True True
buying buy    xxxx True False
U.K. U.K.    X.X. False False
startup startup    xxxx True False
for for    xxx True True
$ $    $ False False
1 1    d False False
billion billion    xxxx True False


In [20]:
docs = nlp(" ".join(reviews["Coco"]))

In [22]:
new_docs = [token.lower().lemma_ for token in docs]

AttributeError: 'str' object has no attribute 'lemma_'

In [17]:
docs[:40]

Im Mexican and all i can say is Thanks you Pizax,I just saw this movie and i just remembered all my childhood with my grandparents, Mexico was represented beautifully,the music, the colors.

In [18]:
new_docs[-1]

'!'

In [ ]:
stemming, lemmatization

In [ ]:
.replace("\", "")